# Getting Started

To get the best result, you can follow my preivous dicussion about [How to win object detection competetion](https://www.kaggle.com/c/global-wheat-detection/discussion/232550#1273363) where i got a gold medal.

And [This notebook](https://www.kaggle.com/espsiyam/yolov5-ensemble-tta-transfer-learning-hpt)

In [ ]:
# Importing Required packages
import os
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Image, clear_output
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))
clear_output()

In [ ]:
#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
# Cloning the repo and installing requiements
!git clone https://github.com/ultralytics/yolov5.git
!mv ./yolov5/* ./
!pip install -r requirements.txt
clear_output()

# Preparing the dataset

In [ ]:
# Copying the dataset to working directory
!mkdir Dataset
!cp ../input/covid19-detection-for-yolov5-siimfisabiorsna/Covid19 -r ./Dataset
clear_output()

In [ ]:
len(os.listdir('./Dataset/Covid19/images'))

In [ ]:
%%writetemplate ./split_dataset.py
from utils.datasets import * 
autosplit('./Dataset/Covid19', weights=(0.8, 0.2, 0.0))

Sptting the dataset for training and validation using datasets from yolov5 repo

In [ ]:
!python split_dataset.py

In [ ]:
os.listdir('./Dataset/Covid19')

In [ ]:
!mkdir DataFile

Defining the dataset

In [ ]:
%%writetemplate ./DataFile/data.yaml

train: ./Dataset/Covid19/autosplit_train.txt
val: ./Dataset/Covid19/autosplit_val.txt

nc: 4
names: ['Negative for Pneumonia', 'Typical Appearance',
        'Indeterminate Appearance', 'Atypical Appearance']

# Model Declaration

you can find all the models [here](https://github.com/ultralytics/yolov5/tree/master/models) and cutomize the classes accordingly.

In [ ]:
%%writetemplate ./models/custom_yolov5x.yaml


# parameters
nc: 4  # number of classes
depth_multiple: 1.33  # model depth multiple
width_multiple: 1.25  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, C3, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, C3, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, C3, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, C3, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

In [ ]:
# I've used wandb before, I don't want it now. So might not need to run this cell
!wandb off

# Training the model

In [ ]:
%%time

!python train.py --img-size 602 --batch 10 --epochs 40 --data './DataFile/data.yaml' --cfg ./models/custom_yolov5x.yaml --weights yolov5x.pt --name experiment2  --cache

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs

# Analyze the result

In [ ]:
os.listdir('./runs/train/experiment2/')

In [ ]:
Image('./runs/train/experiment2/results.png')

In [ ]:
Image('./runs/train/experiment2/confusion_matrix.png',width=400)

In [ ]:
Image('./runs/train/experiment2/test_batch1_labels.jpg')

In [ ]:
Image('./runs/train/experiment2/test_batch1_pred.jpg')

In [ ]:
os.listdir('./Dataset/Covid19')

# Detecting the classes 

In [ ]:
!python detect.py --img-size 602  --conf 0.1 --source ../input/covid19-detection-for-yolov5-siimfisabiorsna/Covid19/images/1000_0.jpg --weights ./runs/train/experiment2/weights/best.pt

In [ ]:
Image('./runs/detect/exp/1000_0.jpg')